# Canteen Menu Optimizer - Fixed Version

This notebook contains the corrected age processing code and other improvements.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Load dataset with error handling
try:
    df = pd.read_csv("data.csv")
    print(f"✅ Dataset loaded successfully!")
    print(f"📏 Dataset shape: {df.shape}")
    print(f"📋 Total columns: {len(df.columns)}")
    print(f"📝 First few columns: {list(df.columns[:10])}")
except FileNotFoundError:
    print("❌ Error: data.csv file not found!")
    print("Please make sure the data.csv file is in the same directory as this notebook.")
    raise
except Exception as e:
    print(f"❌ Error loading dataset: {str(e)}")
    raise

In [ ]:
# Clean column names
def clean_col(c):
    return (str(c).strip()
            .replace("\n", " ")
            .replace("  ", " ")
            .lower()
            .replace(" ", "_")
            .replace("(", "")
            .replace(")", "")
            .replace("/", "_"))

df.columns = [clean_col(c) for c in df.columns]
print("Column names cleaned successfully!")

In [ ]:
# FIXED: Age processing - corrected the syntax error
if "age" in df.columns:
    print(f"Processing age column...")
    print(f"Original age sample: {df['age'].head(3).tolist()}")
    
    # Extract numeric age values - fix the DataFrame indexing issue
    age_extracted = df["age"].astype(str).str.extract(r"(\d+)", expand=False)
    
    # Convert to numeric with proper error handling
    df["age"] = pd.to_numeric(age_extracted, errors="coerce")
    
    # Convert to nullable integer type to handle NaN values
    df["age"] = df["age"].astype("Int64")
    
    print(f"Processed age sample: {df['age'].head(3).tolist()}")
    print(f"Age column processed. Range: {df['age'].min()} - {df['age'].max()}")
    print(f"Missing age values: {df['age'].isna().sum()}")
else:
    print("Age column not found in dataset")

In [ ]:
# Process height and weight columns with improved error handling
print("Processing height and weight columns...")

# Handle different possible column names
height_col = "height_cm" if "height_cm" in df.columns else "height"
weight_col = "weight_kg" if "weight_kg" in df.columns else "weight"

print(f"Looking for height column: {height_col}")
print(f"Looking for weight column: {weight_col}")

for col in (height_col, weight_col):
    if col in df.columns:
        print(f"\nProcessing {col}...")
        original_sample = df[col].head(3).tolist()
        print(f"Original {col} sample: {original_sample}")
        
        # Clean numeric data by removing non-numeric characters
        cleaned_values = df[col].astype(str).str.replace(r"[^\d\.]", "", regex=True)
        df[col] = pd.to_numeric(cleaned_values, errors="coerce")
        
        cleaned_sample = df[col].head(3).tolist()
        print(f"Cleaned {col} sample: {cleaned_sample}")
        print(f"{col} processed. Range: {df[col].min():.1f} - {df[col].max():.1f}")
        print(f"Missing {col} values: {df[col].isna().sum()}")
    else:
        print(f"❌ {col} column not found in dataset")

In [ ]:
# Calculate BMI with improved validation
print("\nCalculating BMI...")

if height_col in df.columns and weight_col in df.columns:
    # Check if we have valid data
    valid_height = df[height_col].notna().sum()
    valid_weight = df[weight_col].notna().sum()
    
    print(f"Valid height values: {valid_height}/{len(df)}")
    print(f"Valid weight values: {valid_weight}/{len(df)}")
    
    if valid_height > 0 and valid_weight > 0:
        # Convert height to meters if it's in centimeters
        if "cm" in height_col:
            height_in_m = df[height_col] / 100
            print(f"Converting height from cm to meters")
        else:
            height_in_m = df[height_col]
            print(f"Using height in meters (assuming input is in meters)")
        
        # Calculate BMI: weight(kg) / height(m)²
        # Add validation to avoid division by zero
        df["bmi"] = df[weight_col] / (height_in_m ** 2)
        
        # Handle extreme values (likely data errors)
        df.loc[df["bmi"] > 100, "bmi"] = np.nan  # Extremely high BMI (likely error)
        df.loc[df["bmi"] < 10, "bmi"] = np.nan   # Extremely low BMI (likely error)
        
        df["bmi"] = df["bmi"].round(2)
        
        valid_bmi = df["bmi"].notna().sum()
        print(f"✅ BMI calculated for {valid_bmi} records")
        print(f"BMI range: {df['bmi'].min():.1f} - {df['bmi'].max():.1f}")
        print(f"Average BMI: {df['bmi'].mean():.2f}")
        print(f"Missing BMI values: {df['bmi'].isna().sum()}")
    else:
        print("❌ Cannot calculate BMI - insufficient valid height or weight data")
else:
    print("❌ Cannot calculate BMI - height or weight column missing")

In [ ]:
# Process target variable - handle different possible names
target_col = "dietary_pref" if "dietary_pref" in df.columns else "dietary_preference"

if target_col in df.columns:
    # Clean and standardize target values
    df[target_col] = df[target_col].astype(str).str.strip().str.title()
    df = df[df[target_col].notna() & (df[target_col] != "Nan")]
    
    print(f"Target variable ({target_col}) distribution:")
    print(df[target_col].value_counts())
else:
    print(f"Target column not found. Available columns: {list(df.columns)}")

In [ ]:
# Process additional numeric columns
numeric_cols = ["spice_tolerance", "sweet_tooth_level", "eating_out_per_week", "food_budget_per_meal"]

for col in numeric_cols:
    if col in df.columns:
        # Clean and convert to numeric
        df[col] = pd.to_numeric(
            df[col].astype(str).str.replace(r"[^\d\.]", "", regex=True), 
            errors="coerce"
        )
        print(f"{col} processed. Range: {df[col].min()} - {df[col].max()}")
    else:
        print(f"{col} column not found")

In [ ]:
# Process cuisine columns
cuisine_cols = ["cuisine_top1", "cuisine_top2", "cuisine_top3"]

for col in cuisine_cols:
    if col in df.columns:
        # Clean cuisine data
        df[col] = df[col].astype(str).replace("nan", pd.NA).replace("", pd.NA).str.strip()
        print(f"{col} processed. Unique values: {df[col].nunique()}")

# Count number of cuisines recorded per person
available_cuisine_cols = [c for c in cuisine_cols if c in df.columns]
if available_cuisine_cols:
    df["num_cuisines_recorded"] = df[available_cuisine_cols].notna().sum(axis=1)
    print(f"\nCuisine count feature created. Range: {df['num_cuisines_recorded'].min()} - {df['num_cuisines_recorded'].max()}")

In [ ]:
# Create interaction features (advanced feature engineering)
if "spice_tolerance" in df.columns and "sweet_tooth_level" in df.columns:
    # Create spice-sweet interaction feature
    df["spice_sweet_interaction"] = df["spice_tolerance"] * df["sweet_tooth_level"]
    print("Spice-sweet interaction feature created")

# Remove duplicates
original_shape = df.shape
df = df.drop_duplicates().reset_index(drop=True)
duplicates_removed = original_shape[0] - df.shape[0]
print(f"\nRemoved {duplicates_removed} duplicate rows")

In [ ]:
# Final data summary
print(f"\n=== FINAL DATASET SUMMARY ===")
print(f"Shape: {df.shape}")
print(f"Missing values: {df.isnull().sum().sum()}")
print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024:.1f} KB")

if target_col in df.columns:
    print(f"\nTarget variable distribution:")
    target_dist = df[target_col].value_counts()
    for diet_type, count in target_dist.items():
        percentage = (count / len(df)) * 100
        print(f"  • {diet_type}: {count} ({percentage:.1f}%)")

print(f"\n=== DATA PROCESSING COMPLETE ===")
print(f"✅ All errors have been fixed")
print(f"✅ Dataset is ready for machine learning")

# Display first few rows
print(f"\nFirst few rows:")
df.head()

In [ ]:
# Validation and final checks
print(f"\n=== VALIDATION CHECKS ===")

# Check for critical errors
errors = []

if df.empty:
    errors.append("Dataset is empty")

if 'age' in df.columns and df['age'].isna().all():
    errors.append("All age values are missing")

if target_col in df.columns and df[target_col].isna().all():
    errors.append("All target values are missing")

if errors:
    print("❌ CRITICAL ERRORS FOUND:")
    for error in errors:
        print(f"  • {error}")
else:
    print("✅ All validation checks passed!")
    
    # Save the cleaned dataset
    output_filename = "canteen_data_cleaned_fixed.csv"
    try:
        df.to_csv(output_filename, index=False)
        print(f"\n💾 Cleaned dataset saved as: {output_filename}")
        print(f"📊 Dataset is ready for machine learning modeling!")
        print(f"\n🎉 SUCCESS: All errors have been fixed and data is ready to use!")
    except Exception as e:
        print(f"❌ Error saving file: {str(e)}")